In [ ]:
!pip install opendatasets

In [ ]:
import opendatasets as od
import pandas as pd

In [ ]:
od.download("https://www.kaggle.com/datasets/naurosromim/bdshs/data")

100%|██████████| 2.23M/2.23M [00:00<00:00, 3.74MB/s]

In [ ]:
!pip install transformers
!pip install torch
!pip install datasets


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.6 MB/s eta 0:00:00


In [ ]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import AdamW, get_linear_schedule_with_warmup
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from datasets import load_dataset
import pandas as pd
import re

In [ ]:
def remove_emojis(text):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"
                               u"\U0001F300-\U0001F5FF"
                               u"\U0001F680-\U0001F6FF"
                               u"\U0001F700-\U0001F77F"
                               u"\U0001F780-\U0001F7FF"
                               u"\U0001F800-\U0001F8FF"
                               u"\U0001F900-\U0001F9FF"
                               u"\U0001FA00-\U0001FA6F"
                               u"\U0001FA70-\U0001FAFF"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

In [ ]:
df_init= pd.read_csv("/content/bdshs/train.csv")

In [ ]:
# Take 12k rows randomly from the train dataset
df = df_init.sample(n=12000, random_state=42)

# Reset the index to start from 0
df = df.reset_index(drop=True)

# Display the first few rows of the new dataset
print(df.head())

                                            sentence target     type  \
0  এখানে অবশ্যই মুন্নি জড়িত সন্ত্রাসীরা ওকে এভাব...    NaN      NaN   
1               আসহাই কাকে বলে হেপিকে দেখলে ভুজা জাই    NaN      NaN   
2  অপেক্ষায় ছিলাম এই রোস্টের জন্যে ধন্যবাদ তাহসিন...    NaN      NaN   
3  উউছিত কাজ করছে সরকার আইন সবার জন্য সমান তুমি স...    NaN      NaN   
4  তুমি শালা আর কিছু পেলেনা তাইনা কত চাইছিলা যা দ...   male  slander   

   hate speech  
0            0  
1            0  
2            0  
3            0  
4            1  


In [ ]:
# Specify columns to drop
columns_to_drop = ['target', 'type']

# Drop the specified columns
# train_data = random_train_data.drop(columns=columns_to_drop)
df = df.drop(columns=columns_to_drop)

# Display the DataFrame after dropping columns
print("\nDataFrame after dropping specified columns:")
print(df)


DataFrame after dropping specified columns:
                                                sentence  hate speech
0      এখানে অবশ্যই মুন্নি জড়িত সন্ত্রাসীরা ওকে এভাব...            0
1                   আসহাই কাকে বলে হেপিকে দেখলে ভুজা জাই            0
2      অপেক্ষায় ছিলাম এই রোস্টের জন্যে ধন্যবাদ তাহসিন...            0
3      উউছিত কাজ করছে সরকার আইন সবার জন্য সমান তুমি স...            0
4      তুমি শালা আর কিছু পেলেনা তাইনা কত চাইছিলা যা দ...            1
...                                                  ...          ...
11995  নাস্তিক পীর ভন্ড পীর থেকে হে দেশ বাসী সাবধান।ত...            1
11996  যে এইটা ভিডিও করেছে সে ত্য এই পোলার থেকেও বড় ব...            1
11997                     সেফাত উল্লাকে কেউ মেনশন দেন তো            0
11998  সালা আজকালকার মেয়ে গুলাকে বিয়ে করাই ঠিক না ন...            1
11999  পাপনের পদত্যাগ চাই ভাই আমরা।হ্যা ভাই সাকিব না ...            0

[12000 rows x 2 columns]


In [ ]:
# Apply the remove_emojis function to the 'Text' column
df['sentence'] = df['sentence'].apply(remove_emojis)

# Display the modified DataFrame
print("\nDataFrame after removing emojis:")
print(df)



DataFrame after removing emojis:
                                                sentence  hate speech
0      এখানে অবশ্যই মুন্নি জড়িত সন্ত্রাসীরা ওকে এভাব...            0
1                   আসহাই কাকে বলে হেপিকে দেখলে ভুজা জাই            0
2      অপেক্ষায় ছিলাম এই রোস্টের জন্যে ধন্যবাদ তাহসিন...            0
3      উউছিত কাজ করছে সরকার আইন সবার জন্য সমান তুমি স...            0
4      তুমি শালা আর কিছু পেলেনা তাইনা কত চাইছিলা যা দ...            1
...                                                  ...          ...
11995  নাস্তিক পীর ভন্ড পীর থেকে হে দেশ বাসী সাবধান।ত...            1
11996  যে এইটা ভিডিও করেছে সে ত্য এই পোলার থেকেও বড় ব...            1
11997                     সেফাত উল্লাকে কেউ মেনশন দেন তো            0
11998  সালা আজকালকার মেয়ে গুলাকে বিয়ে করাই ঠিক না ন...            1
11999  পাপনের পদত্যাগ চাই ভাই আমরা।হ্যা ভাই সাকিব না ...            0

[12000 rows x 2 columns]


In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

In [ ]:
# Data preprocessing
X = df['sentence'].values
y = df['hate speech'].values

In [ ]:
# Tokenize and pad sequences
max_words = 50000  # Adjust based on your dataset
max_len = 128     # Adjust based on your dataset
tokenizer = Tokenizer(num_words=max_words, oov_token='<OOV>')
tokenizer.fit_on_texts(X)
X_seq = tokenizer.texts_to_sequences(X)
X_pad = pad_sequences(X_seq, maxlen=max_len)

In [ ]:
# Encode labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)


In [ ]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_pad, y_encoded, test_size=0.2, random_state=42)


In [ ]:
# Build the LSTM model
embedding_dim = 50  # Adjust based on your dataset
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=max_len))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
# Train the model
epochs = 10  # Adjust based on your dataset
batch_size = 32  # Adjust based on your dataset
history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=0.2)

Epoch 1/10
240/240 [==============================] - 3s 13ms/step - loss: 0.0036 - accuracy: 0.9993 - val_loss: 1.3072 - val_accuracy: 0.7974
Epoch 2/10
240/240 [==============================] - 2s 10ms/step - loss: 7.1159e-04 - accuracy: 1.0000 - val_loss: 1.3223 - val_accuracy: 0.7922
Epoch 3/10
240/240 [==============================] - 2s 10ms/step - loss: 5.0906e-04 - accuracy: 0.9999 - val_loss: 1.5841 - val_accuracy: 0.7979
Epoch 4/10
240/240 [==============================] - 2s 9ms/step - loss: 2.7656e-04 - accuracy: 0.9999 - val_loss: 1.4778 - val_accuracy: 0.7911
Epoch 5/10
240/240 [==============================] - 3s 12ms/step - loss: 2.7008e-04 - accuracy: 0.9999 - val_loss: 1.5654 - val_accuracy: 0.7948
Epoch 6/10
240/240 [==============================] - 3s 12ms/step - loss: 1.7837e-04 - accuracy: 1.0000 - val_loss: 1.5659 - val_accuracy: 0.7953
Epoch 7/10
240/240 [==============================] - 2s 9ms/step - loss: 1.8549e-04 - accuracy: 1.0000 - val_loss: 1.5998 

In [ ]:
# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss:.4f}, Test Accuracy: {accuracy:.4f}')


75/75 [==============================] - 0s 5ms/step - loss: 1.3697 - accuracy: 0.7925
Test Loss: 1.3697, Test Accuracy: 0.7925


In [ ]:
# Example of making predictions
sample_text = ["আইসিসিকে এই তথ্য দিলো কে?শুওরের বাচ্চা পাপন"]
sample_seq = tokenizer.texts_to_sequences(sample_text)
sample_pad = pad_sequences(sample_seq, maxlen=max_len)
prediction = model.predict(sample_pad)
predicted_label = "Hate Speech" if prediction[0] >= 0.5 else "Not Hate Speech"
print(f'Predicted Label: {predicted_label}')

1/1 [==============================] - 0s 18ms/step
Predicted Label: Hate Speech


In [ ]:
# Save the model
model.save('hate_speech.h5')  # Save the model as an HDF5 file


In [ ]:
import pickle

# Save the tokenizer to a file using pickle
tokenizer_save_path = 'tokenizer.pickle'
with open(tokenizer_save_path, 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
